# Search Report

In [ ]:
# import sys
# import os

# submit_dir=os.environ['SLURM_SUBMIT_DIR']
# sys.path.append(submit_dir)

In [ ]:
import matplotlib.pylab as plt
from IPython.display import display, Markdown, Latex

import re

import imageio
import numpy as np

import CocoRun
from see import Segmentors
from see import JupyterGUI

import inspect
import argparse

outputfolder = './'

images, masks, outputs = CocoRun.getCocoFolderLists(outputfolder)

In [ ]:
#Fitness for all files
file = open(outputfolder+'params.txt','r')
fitness = []
filelines = file.readlines()
params = filelines[0]
for line in filelines[1:]:
    match = re.search(' ', line)
    fitness.append(float(line[:match.start()]))
print(params)

In [ ]:
seg = Segmentors.algoFromParams(eval(params))

code = "```python\n"
code += inspect.getsource(seg.evaluate)
code += "```\n"

display(Markdown(code))

In [ ]:
fitness = np.array(fitness)
total = len(fitness)
sorted_fitness = fitness.copy()
sorted_fitness.sort()

fivehundred_err = len(fitness[fitness < 500]) / total * 100
print(f"This parameter got {fivehundred_err}% on all the Coco files\n")

In [ ]:
plt.hist(fitness[fitness < 500], 50);
plt.xlabel('fitness values')
plt.ylabel('image count');

In [ ]:
###How fitness changes over time
file = open(f"{outputfolder}search.out","r") 
match = ''
tfitness = []
for line in file.readlines():
    if re.search('#BEST', line):
        match = re.search(" \- \[", line)
        tfitness.append(float(line[8:match.start()]))
file.close()

plt.plot(tfitness)
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.title('Evolution of the fitness');

In [ ]:
#for imagename, maskname, outputname in zip(images, masks, output):
index = 10

imagename = images[index]
maskname = masks[index]
outputname = outputs[index]

print("\n")

image = imageio.imread(imagename)
mask = imageio.imread(maskname)
output = imageio.imread(outputname)

JupyterGUI.showthree(image,mask,output)
print(f"{image} {mask} {output}")
